In [1]:
import pandas as pd
import os
import keras
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from keras.utils import np_utils
from keras.models import Sequential, Model
from keras.layers.merge import concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import  Input, Dense, Conv2D, Dropout, Flatten, MaxPooling2D, LSTM
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_data(dataset_path='./npy_files'):
    X = np.load(os.path.join(dataset_path,'X.npy'))
    y = np.load(os.path.join(dataset_path,'y.npy'))
    folds = np.load(os.path.join(dataset_path,'folds.npy'))
    feature_class = np.load(os.path.join(dataset_path, 'feature_class.npy'))
    facs = pd.read_csv('save_data/npy_files/facs_multi.csv',index_col=None)

    # rescale [0,255] -> [0,2]    
    X = X.astype('float32')/255*2


    return X, y, folds, feature_class, facs

In [3]:
X, y, folds, feature_class, facs = load_data('./save_data/npy_files/')
del facs['Unnamed: 0']
del facs['10']
del facs['26']
del facs['0']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
feature_class_single = np.zeros((1307, 4040))
for index, inst in enumerate(feature_class):
    feature_class_single[index] = inst[0]

In [6]:
feature_train, feature_test, facs_train, facs_test = \
        train_test_split(feature_class_single, facs, test_size=0.2)

In [7]:
feature_class_single

array([[0.00011116, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [8]:
facs

,1,2,4,5,6,7,9,12,15,17,20,23,24,25,27
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
6,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
7,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [9]:
facs

,1,2,4,5,6,7,9,12,15,17,20,23,24,25,27
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
6,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
7,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0


In [12]:
emotion_au = np.zeros((7,15))
for index, facs_value in facs.iterrows():
    if y[index] == 0:
        continue
    
    emotion_au[y[index]-1] += facs_value.astype(int)

In [38]:
emotion_au_df = pd.DataFrame(emotion_au, columns=facs.columns)
emotion_au_df = emotion_au_df.astype(int)

In [43]:
emotion_au_df

,1,2,4,5,6,7,9,12,15,17,20,23,24,25,27
0,0,0,120,18,24,96,9,3,9,117,0,108,99,0,0
1,3,3,3,0,0,0,0,15,12,15,0,3,6,0,0
2,3,0,108,0,54,99,174,6,6,120,0,6,21,27,0
3,66,30,63,48,9,18,0,6,0,9,75,0,0,69,0
4,0,0,0,0,198,21,0,201,0,0,3,0,0,201,0
5,75,18,66,0,0,3,0,0,66,78,0,9,3,0,0
6,243,243,3,210,0,0,0,9,3,0,3,3,0,246,216


In [35]:
au_num_by_code = (facs.apply(sum).astype(float))

In [44]:
emotion_au_percent

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]

In [56]:
emotion_au_percent = []
for au_row in emotion_au_df.iterrows():
    tmp_row = []
    for au_code, au_num in  zip(au_num_by_code.index, au_num_by_code):
        tmp_row.append(au_row[1][au_code] / au_num)
    emotion_au_percent.append(tmp_row)
emotion_au_percent = pd.DataFrame(emotion_au_percent, columns=facs.columns)
emotion_au_percent = emotion_au_percent.applymap(lambda x: (round(x*100,2)))

In [57]:
emotion_au_percent

,1,2,4,5,6,7,9,12,15,17,20,23,24,25,27
0,0.00,0.00,33.06,6.52,8.42,40.51,4.92,1.25,9.38,34.51,0.00,83.72,76.74,0.00,0.0
1,0.77,1.02,0.83,0.00,0.00,0.00,0.00,6.25,12.50,4.42,0.00,2.33,4.65,0.00,0.0
2,0.77,0.00,29.75,0.00,18.95,41.77,95.08,2.50,6.25,35.40,0.00,4.65,16.28,4.97,0.0
3,16.92,10.20,17.36,17.39,3.16,7.59,0.00,2.50,0.00,2.65,92.59,0.00,0.00,12.71,0.0
4,0.00,0.00,0.00,0.00,69.47,8.86,0.00,83.75,0.00,0.00,3.70,0.00,0.00,37.02,0.0
5,19.23,6.12,18.18,0.00,0.00,1.27,0.00,0.00,68.75,23.01,0.00,6.98,2.33,0.00,0.0
6,62.31,82.65,0.83,76.09,0.00,0.00,0.00,3.75,3.12,0.00,3.70,2.33,0.00,45.30,100.0


In [58]:
emotion_au_percent_str = emotion_au_percent.applymap(lambda x: str(x)+'%')

In [59]:
emotion_au_percent_str

,1,2,4,5,6,7,9,12,15,17,20,23,24,25,27
0,0.0%,0.0%,33.06%,6.52%,8.42%,40.51%,4.92%,1.25%,9.38%,34.51%,0.0%,83.72%,76.74%,0.0%,0.0%
1,0.77%,1.02%,0.83%,0.0%,0.0%,0.0%,0.0%,6.25%,12.5%,4.42%,0.0%,2.33%,4.65%,0.0%,0.0%
2,0.77%,0.0%,29.75%,0.0%,18.95%,41.77%,95.08%,2.5%,6.25%,35.4%,0.0%,4.65%,16.28%,4.97%,0.0%
3,16.92%,10.2%,17.36%,17.39%,3.16%,7.59%,0.0%,2.5%,0.0%,2.65%,92.59%,0.0%,0.0%,12.71%,0.0%
4,0.0%,0.0%,0.0%,0.0%,69.47%,8.86%,0.0%,83.75%,0.0%,0.0%,3.7%,0.0%,0.0%,37.02%,0.0%
5,19.23%,6.12%,18.18%,0.0%,0.0%,1.27%,0.0%,0.0%,68.75%,23.01%,0.0%,6.98%,2.33%,0.0%,0.0%
6,62.31%,82.65%,0.83%,76.09%,0.0%,0.0%,0.0%,3.75%,3.12%,0.0%,3.7%,2.33%,0.0%,45.3%,100.0%


In [61]:
emotion_au_df_str = emotion_au_df
emotion_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sadness',
                          'Surprise', 'Contempt']
emotion_au_df_str.index = emotion_list

In [62]:
emotion_au_df_str

,1,2,4,5,6,7,9,12,15,17,20,23,24,25,27
Angry,0,0,120,18,24,96,9,3,9,117,0,108,99,0,0
Disgust,3,3,3,0,0,0,0,15,12,15,0,3,6,0,0
Fear,3,0,108,0,54,99,174,6,6,120,0,6,21,27,0
Happy,66,30,63,48,9,18,0,6,0,9,75,0,0,69,0
Sadness,0,0,0,0,198,21,0,201,0,0,3,0,0,201,0
Surprise,75,18,66,0,0,3,0,0,66,78,0,9,3,0,0
Contempt,243,243,3,210,0,0,0,9,3,0,3,3,0,246,216


In [63]:
emotion_au_percent_str.index = emotion_list

In [64]:
emotion_au_percent_str

,1,2,4,5,6,7,9,12,15,17,20,23,24,25,27
Angry,0.0%,0.0%,33.06%,6.52%,8.42%,40.51%,4.92%,1.25%,9.38%,34.51%,0.0%,83.72%,76.74%,0.0%,0.0%
Disgust,0.77%,1.02%,0.83%,0.0%,0.0%,0.0%,0.0%,6.25%,12.5%,4.42%,0.0%,2.33%,4.65%,0.0%,0.0%
Fear,0.77%,0.0%,29.75%,0.0%,18.95%,41.77%,95.08%,2.5%,6.25%,35.4%,0.0%,4.65%,16.28%,4.97%,0.0%
Happy,16.92%,10.2%,17.36%,17.39%,3.16%,7.59%,0.0%,2.5%,0.0%,2.65%,92.59%,0.0%,0.0%,12.71%,0.0%
Sadness,0.0%,0.0%,0.0%,0.0%,69.47%,8.86%,0.0%,83.75%,0.0%,0.0%,3.7%,0.0%,0.0%,37.02%,0.0%
Surprise,19.23%,6.12%,18.18%,0.0%,0.0%,1.27%,0.0%,0.0%,68.75%,23.01%,0.0%,6.98%,2.33%,0.0%,0.0%
Contempt,62.31%,82.65%,0.83%,76.09%,0.0%,0.0%,0.0%,3.75%,3.12%,0.0%,3.7%,2.33%,0.0%,45.3%,100.0%


In [68]:
au_str_list = list(map(lambda x: 'AU '+str(x), facs.columns))

In [69]:
au_str_list

['AU 1',
 'AU 2',
 'AU 4',
 'AU 5',
 'AU 6',
 'AU 7',
 'AU 9',
 'AU 12',
 'AU 15',
 'AU 17',
 'AU 20',
 'AU 23',
 'AU 24',
 'AU 25',
 'AU 27']

In [70]:
emotion_au_percent_str2 = emotion_au_percent_str
emotion_au_percent_str2.columns = au_str_list

In [71]:
emotion_au_percent_str2

,AU 1,AU 2,AU 4,AU 5,AU 6,AU 7,AU 9,AU 12,AU 15,AU 17,AU 20,AU 23,AU 24,AU 25,AU 27
Angry,0.0%,0.0%,33.06%,6.52%,8.42%,40.51%,4.92%,1.25%,9.38%,34.51%,0.0%,83.72%,76.74%,0.0%,0.0%
Disgust,0.77%,1.02%,0.83%,0.0%,0.0%,0.0%,0.0%,6.25%,12.5%,4.42%,0.0%,2.33%,4.65%,0.0%,0.0%
Fear,0.77%,0.0%,29.75%,0.0%,18.95%,41.77%,95.08%,2.5%,6.25%,35.4%,0.0%,4.65%,16.28%,4.97%,0.0%
Happy,16.92%,10.2%,17.36%,17.39%,3.16%,7.59%,0.0%,2.5%,0.0%,2.65%,92.59%,0.0%,0.0%,12.71%,0.0%
Sadness,0.0%,0.0%,0.0%,0.0%,69.47%,8.86%,0.0%,83.75%,0.0%,0.0%,3.7%,0.0%,0.0%,37.02%,0.0%
Surprise,19.23%,6.12%,18.18%,0.0%,0.0%,1.27%,0.0%,0.0%,68.75%,23.01%,0.0%,6.98%,2.33%,0.0%,0.0%
Contempt,62.31%,82.65%,0.83%,76.09%,0.0%,0.0%,0.0%,3.75%,3.12%,0.0%,3.7%,2.33%,0.0%,45.3%,100.0%


In [72]:
emotion_au_df_str2 = emotion_au_df_str
emotion_au_df_str2.columns = au_str_list

In [73]:
emotion_au_df_str2

,AU 1,AU 2,AU 4,AU 5,AU 6,AU 7,AU 9,AU 12,AU 15,AU 17,AU 20,AU 23,AU 24,AU 25,AU 27
Angry,0,0,120,18,24,96,9,3,9,117,0,108,99,0,0
Disgust,3,3,3,0,0,0,0,15,12,15,0,3,6,0,0
Fear,3,0,108,0,54,99,174,6,6,120,0,6,21,27,0
Happy,66,30,63,48,9,18,0,6,0,9,75,0,0,69,0
Sadness,0,0,0,0,198,21,0,201,0,0,3,0,0,201,0
Surprise,75,18,66,0,0,3,0,0,66,78,0,9,3,0,0
Contempt,243,243,3,210,0,0,0,9,3,0,3,3,0,246,216


In [107]:
au_emotion = emotion_au_df_str2.T

In [93]:
emotion_num = np.zeros((7))
for emot in y:
    emotion_num[emot-1] +=1

In [94]:
emotion_num

array([135.,  54., 177.,  75., 207.,  81., 578.])

In [109]:
au_emotion

,Angry,Disgust,Fear,Happy,Sadness,Surprise,Contempt
AU 1,0,3,3,66,0,75,243
AU 2,0,3,0,30,0,18,243
AU 4,120,3,108,63,0,66,3
AU 5,18,0,0,48,0,0,210
AU 6,24,0,54,9,198,0,0
AU 7,96,0,99,18,21,3,0
AU 9,9,0,174,0,0,0,0
AU 12,3,15,6,6,201,0,9
AU 15,9,12,6,0,0,66,3
AU 17,117,15,120,9,0,78,0


In [110]:
au_emotion_percent = []
for au_row in au_emotion.iterrows():
    tmp_row = []
    for index, emotion_code in enumerate(emotion_num):
        tmp_row.append(au_row[1].iloc[index] / emotion_code)
    au_emotion_percent.append(tmp_row)
au_emotion_percent = pd.DataFrame(au_emotion_percent, columns=emotion_list)
au_emotion_percent = au_emotion_percent.applymap(lambda x: (round(x*100,2)))

In [118]:
au_emotion_percent_str = au_emotion_percent
au_emotion_percent_str.index = au_str_list
au_emotion_percent_str = au_emotion_percent_str.applymap(lambda x: str(x)+'%')
au_emotion_percent_str

,Angry,Disgust,Fear,Happy,Sadness,Surprise,Contempt
AU 1,0.0%,5.56%,1.69%,88.0%,0.0%,92.59%,42.04%
AU 2,0.0%,5.56%,0.0%,40.0%,0.0%,22.22%,42.04%
AU 4,88.89%,5.56%,61.02%,84.0%,0.0%,81.48%,0.52%
AU 5,13.33%,0.0%,0.0%,64.0%,0.0%,0.0%,36.33%
AU 6,17.78%,0.0%,30.51%,12.0%,95.65%,0.0%,0.0%
AU 7,71.11%,0.0%,55.93%,24.0%,10.14%,3.7%,0.0%
AU 9,6.67%,0.0%,98.31%,0.0%,0.0%,0.0%,0.0%
AU 12,2.22%,27.78%,3.39%,8.0%,97.1%,0.0%,1.56%
AU 15,6.67%,22.22%,3.39%,0.0%,0.0%,81.48%,0.52%
AU 17,86.67%,27.78%,67.8%,12.0%,0.0%,96.3%,0.0%
